# Create the automl job for object detection

## Connection to ml client 

In [1]:
from lib import get_client
client=get_client()
print(f'workspace name: {client.workspace_name}')

workspace name: azure-ml


##  Create Job

Creation of the input path from the object detection job 

In [8]:
# Name of train data asset 
train_data_asset="train-fashionpedia-detection-automl"
# Name of validation test asset 
validation_data_asset="validation-fashionpedia-detection-automl"
# Name of compute 
compute="gpu01"
# experiment name 
experiment_name="fashionpedia-detection"

In [3]:
train_asset=client.data.get(name=train_data_asset,version="1.0.0")
validation_asset=client.data.get(name=validation_data_asset,version="1.0.0")
print(f'train path {train_asset.path}')
print(f'validation path {validation_asset.path}')

train path azureml://subscriptions/7db1ee9f-abea-4b8b-b5a9-e6e82ecfa19d/resourcegroups/azure-ml/workspaces/azure-ml/datastores/workspaceblobstore/paths/LocalUpload/df97cad8fd2bdbea495f0b7c96c28be9/train/
validation path azureml://subscriptions/7db1ee9f-abea-4b8b-b5a9-e6e82ecfa19d/resourcegroups/azure-ml/workspaces/azure-ml/datastores/workspaceblobstore/paths/LocalUpload/4787a0c625e38021cb682fc763637ebe/validation/


In [4]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

training_data_input = Input(type=AssetTypes.MLTABLE, path=train_asset.path)
validation_data_input = Input(type=AssetTypes.MLTABLE, path=validation_asset.path)
print(f'Train data input: {training_data_input}')
print(f'Validation data input:{validation_data_input} ')

Train data input: {'type': 'mltable', 'path': 'azureml://subscriptions/7db1ee9f-abea-4b8b-b5a9-e6e82ecfa19d/resourcegroups/azure-ml/workspaces/azure-ml/datastores/workspaceblobstore/paths/LocalUpload/df97cad8fd2bdbea495f0b7c96c28be9/train/'}
Validation data input:{'type': 'mltable', 'path': 'azureml://subscriptions/7db1ee9f-abea-4b8b-b5a9-e6e82ecfa19d/resourcegroups/azure-ml/workspaces/azure-ml/datastores/workspaceblobstore/paths/LocalUpload/4787a0c625e38021cb682fc763637ebe/validation/'} 


In [9]:
from azure.ai.ml import automl
image_classification_job = automl.image_object_detection(
       experiment_name=experiment_name,
       compute=compute,
       training_data=training_data_input,
       validation_data=validation_data_input,
       target_column_name="label",
       tags={"fashionpedia": "detection"},
   )


## Hyper Parameters configuration

We can configure some hyperparameter or keep the default value use by azure. 

See image job default values hyperparameters in:

https://learn.microsoft.com/en-gb/azure/machine-learning/reference-automl-images-hyperparameters?view=azureml-api-2

we will just change the number of epochs 

In [10]:
image_classification_job.set_training_parameters(number_of_epochs=8)

## Submit the job
Be sure that compute is running before to submit the job

In [11]:
returned_job=client.create_or_update(image_classification_job)
print(f"Created job: {returned_job}")

Created job: compute: azureml:gpu01
creation_context:
  created_at: '2024-05-19T11:00:39.140576+00:00'
  created_by: bc364512-35e7-4ad5-be27-080788c014f5
  created_by_type: Application
display_name: upbeat_reggae_1707vs71rc
experiment_name: fashionpedia-detection
id: azureml:/subscriptions/7db1ee9f-abea-4b8b-b5a9-e6e82ecfa19d/resourceGroups/azure-ml/providers/Microsoft.MachineLearningServices/workspaces/azure-ml/jobs/upbeat_reggae_1707vs71rc
limits:
  max_concurrent_trials: 1
  max_trials: 1
  timeout_minutes: 10080
log_verbosity: info
name: upbeat_reggae_1707vs71rc
outputs: {}
primary_metric: mean_average_precision
properties:
  azureml.git.dirty: 'True'
  mlflow.source.git.branch: main
resources:
  instance_count: 1
  shm_size: 2g
services:
  Studio:
    endpoint: https://ml.azure.com/runs/upbeat_reggae_1707vs71rc?wsid=/subscriptions/7db1ee9f-abea-4b8b-b5a9-e6e82ecfa19d/resourcegroups/azure-ml/workspaces/azure-ml&tid=5d250ee6-9438-4941-b7c3-dab39198c26a
  Tracking:
    endpoint: azur